In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import functools
import glob
import os
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
start_date = '2023-10-18'
final_date = '2023-10-30'

### EDO DE CUENTA

In [3]:
all_mov = glob.glob(r'C:\Users\Cristian.Aparicio\Documents\CONCILIACIONES\2023\**\EDO CTA BNTE\*.txt', recursive=False)
all_mov

['C:\\Users\\Cristian.Aparicio\\Documents\\CONCILIACIONES\\2023\\01. ENERO\\EDO CTA BNTE\\BNTE 2023.01.02 PT1.txt',
 'C:\\Users\\Cristian.Aparicio\\Documents\\CONCILIACIONES\\2023\\01. ENERO\\EDO CTA BNTE\\BNTE 2023.01.02 PT10.txt',
 'C:\\Users\\Cristian.Aparicio\\Documents\\CONCILIACIONES\\2023\\01. ENERO\\EDO CTA BNTE\\BNTE 2023.01.02 PT11.txt',
 'C:\\Users\\Cristian.Aparicio\\Documents\\CONCILIACIONES\\2023\\01. ENERO\\EDO CTA BNTE\\BNTE 2023.01.02 PT12.txt',
 'C:\\Users\\Cristian.Aparicio\\Documents\\CONCILIACIONES\\2023\\01. ENERO\\EDO CTA BNTE\\BNTE 2023.01.02 PT13.txt',
 'C:\\Users\\Cristian.Aparicio\\Documents\\CONCILIACIONES\\2023\\01. ENERO\\EDO CTA BNTE\\BNTE 2023.01.02 PT14.txt',
 'C:\\Users\\Cristian.Aparicio\\Documents\\CONCILIACIONES\\2023\\01. ENERO\\EDO CTA BNTE\\BNTE 2023.01.02 PT15.txt',
 'C:\\Users\\Cristian.Aparicio\\Documents\\CONCILIACIONES\\2023\\01. ENERO\\EDO CTA BNTE\\BNTE 2023.01.02 PT16.txt',
 'C:\\Users\\Cristian.Aparicio\\Documents\\CONCILIACIONES\\2023\\

# EDO CUENTA COMPLETO

In [4]:
edo_cta = pd.concat([pd.read_csv(f,sep='|',dtype={'Cuenta': object,'Cod. Transac':object})
                     .assign(file_name=os.path.basename(f)) for f in all_mov])
edo_cta.reset_index(inplace=True)
del edo_cta['index']
edo_cta.sort_values(by=['Movimiento'],inplace=True)
edo_cta['Cuenta'] = "'"+edo_cta['Cuenta']
edo_cta.head()

,Cuenta,Fecha De Operación,Fecha,Referencia,Descripción,Cod. Transac,Sucursal,Depósitos,Retiros,Saldo,Movimiento,Descripción Detallada,Cheque,file_name
0,'0239650517,31/12/2022,02/01/2023,"400,006.00",TRASPASO A CTA : 0847514546,504,4264,$0.00,"$2,984.90","$756,375,719.65",27454486,"IVA:00000000.00 ,A CUENTA 00400847514546,...",NaN,BNTE 2023.01.02 PT1.txt
1,'0239650517,31/12/2022,02/01/2023,"400,011.00",TRASPASO A CTA : 1191338059,504,4264,$0.00,$115.47,"$756,375,604.18",27454487,"IVA:00000000.00 ,A CUENTA 1191338059, CLI...",NaN,BNTE 2023.01.02 PT1.txt
2,'0239650517,31/12/2022,02/01/2023,"400,381.00",TRASPASO A CTA : 0854271883,504,4264,$0.00,"$13,238.08","$756,362,366.10",27454488,"IVA:00000000.00 ,A CUENTA 00100854271883,...",NaN,BNTE 2023.01.02 PT1.txt
3,'0239650517,31/12/2022,02/01/2023,"400,431.00",TRASPASO A CTA : 0664258739,504,4264,$0.00,"$2,625.58","$756,359,740.52",27454489,"IVA:00000000.00 ,A CUENTA 0664258739, CLI...",NaN,BNTE 2023.01.02 PT1.txt
4,'0239650517,31/12/2022,02/01/2023,"400,461.00",TRASPASO A CTA : 1018461782,504,4264,$0.00,"$1,993.14","$756,357,747.38",27454490,"IVA:00000000.00 ,A CUENTA 1018461782, CLI...",NaN,BNTE 2023.01.02 PT1.txt


In [5]:
edo_cta.shape

(12876303, 14)

In [6]:
split_fecha_operacion = edo_cta["Fecha De Operación"].str.split("/",n=3,expand=True)
split_fecha_operacion.head()

,0,1,2
0,31,12,2022
1,31,12,2022
2,31,12,2022
3,31,12,2022
4,31,12,2022


In [7]:
edo_cta["Day_operacion"] = split_fecha_operacion[0] 
edo_cta["Month_operacion"] = split_fecha_operacion[1]
edo_cta['Year_operacion'] = split_fecha_operacion[2]

In [8]:
split_fecha_banco = edo_cta["Fecha"].str.split("/",n=3,expand=True)
split_fecha_banco.head()

,0,1,2
0,02,01,2023
1,02,01,2023
2,02,01,2023
3,02,01,2023
4,02,01,2023


In [9]:
edo_cta["Day_banco"] = split_fecha_banco[0] 
edo_cta["Month_banco"] = split_fecha_banco[1]
edo_cta['Year_banco'] = split_fecha_banco[2]

In [10]:
edo_cta['Fecha_operacion'] = edo_cta['Year_operacion']+'-'+edo_cta['Month_operacion']+'-'+edo_cta['Day_operacion']
edo_cta['Fecha_banco'] = edo_cta['Year_banco']+'-'+edo_cta['Month_banco']+'-'+edo_cta['Day_banco']

In [11]:
edo_cta['Fecha_operacion'] = pd.to_datetime(edo_cta['Fecha_operacion'])
edo_cta['Fecha_banco'] = pd.to_datetime(edo_cta['Fecha_banco'])

In [12]:
edo_cta.loc[edo_cta['Descripción'].str.contains('08343798'), "Tipo"] = 'XPAY'
edo_cta.loc[edo_cta['Descripción'].str.contains('08380402'), "Tipo"] = 'MIT-3DS'
edo_cta.loc[(~edo_cta['Descripción'].str.contains('08343798')) & (~edo_cta['Descripción'].str.contains('08380402')), "Tipo"] = 'REGULAR TXN'

In [13]:
edo_cta.Depósitos = [x.strip('$') for x in edo_cta.Depósitos]
edo_cta.Retiros = [x.strip('$') for x in edo_cta.Retiros]
edo_cta.Saldo = [x.strip('$') for x in edo_cta.Saldo]

In [14]:
edo_cta['Depósitos'] = edo_cta['Depósitos'].str.replace(',','').astype(float)
edo_cta['Retiros'] = edo_cta['Retiros'].str.replace(',','').astype(float)
edo_cta['Saldo'] = edo_cta['Saldo'].str.replace(',','').astype(float)

ValueError: could not convert string to float: '($2371.71)'

In [15]:
edo_cta.loc[edo_cta['Cod. Transac'] == '060', 'Afiliacion_1'] = edo_cta['Descripción'].str[-9:]
edo_cta.loc[edo_cta['Cod. Transac'] == '061', 'Afiliacion_1'] = edo_cta['Descripción'].str[-9:]
edo_cta.loc[edo_cta['Cod. Transac'] == '062', 'Afiliacion_1'] = edo_cta['Descripción'].str[-9:]
edo_cta.loc[edo_cta['Cod. Transac'] == '600', 'Afiliacion_1'] = edo_cta['Descripción'].str[-9:]
edo_cta.loc[edo_cta['Cod. Transac'] == '601', 'Afiliacion_1'] = edo_cta['Descripción'].str[-9:]
edo_cta.loc[edo_cta['Cod. Transac'] == '602', 'Afiliacion_1'] = edo_cta['Descripción'].str[-9:]
edo_cta.loc[(edo_cta['Cod. Transac'] == '602') &
            (edo_cta['Descripción Detallada'].str.contains('CONTRACARGO')),
            'Afiliacion_1'] = edo_cta['Descripción'].str[-8:]
edo_cta['len'] = edo_cta['Afiliacion_1'].str.len()
edo_cta.loc[(edo_cta['Cod. Transac'] == '060') & (~edo_cta['Descripción Detallada'].str.contains('REVERSO CONTRACARGO', na=False)),
            'Afiliacion_final'] = edo_cta['Afiliacion_1'].str[:8].replace(' ', '',regex=True)
edo_cta.loc[(edo_cta['Cod. Transac'] == '060') & (edo_cta['Descripción Detallada'].str.contains('REVERSO CONTRACARGO')),
            'Afiliacion_final'] = edo_cta['Afiliacion_1'].replace(' ', '',regex=True)
edo_cta.loc[(edo_cta['Cod. Transac'] == '060') & (edo_cta['Afiliacion_1'].str.contains('C')),
            'Tipo_operacion'] = edo_cta['Afiliacion_1'].str[-1:].replace(' ', '',regex=True)
edo_cta.loc[(edo_cta['Cod. Transac'] == '060') & (edo_cta['Afiliacion_1'].str.contains('D')),
            'Tipo_operacion'] = edo_cta['Afiliacion_1'].str[-1:].replace(' ', '',regex=True)
#edo_cta.loc[edo_cta['len'] == 9, 'Afiliacion_final'] = edo_cta['Afiliacion_1'].str[-8:]
# edo_cta.loc[edo_cta['len'] == 8, 'Afiliacion_final'] = edo_cta['Afiliacion_1'].str[-7:]
# edo_cta.loc[edo_cta['len'] == 9, 'Tipo_operacion'] = edo_cta['Afiliacion_1'].str[-1:]
edo_cta.head()

,Cuenta,Fecha De Operación,Fecha,Referencia,Descripción,Cod. Transac,Sucursal,Depósitos,Retiros,Saldo,...,Day_banco,Month_banco,Year_banco,Fecha_operacion,Fecha_banco,Tipo,Afiliacion_1,len,Afiliacion_final,Tipo_operacion
0,'0239650517,31/12/2022,02/01/2023,"400,006.00",TRASPASO A CTA : 0847514546,504,4264,0.00,"2,984.90","756,375,719.65",...,02,01,2023,2022-12-31,2023-01-02,REGULAR TXN,NaN,NaN,NaN,NaN
1,'0239650517,31/12/2022,02/01/2023,"400,011.00",TRASPASO A CTA : 1191338059,504,4264,0.00,115.47,"756,375,604.18",...,02,01,2023,2022-12-31,2023-01-02,REGULAR TXN,NaN,NaN,NaN,NaN
2,'0239650517,31/12/2022,02/01/2023,"400,381.00",TRASPASO A CTA : 0854271883,504,4264,0.00,"13,238.08","756,362,366.10",...,02,01,2023,2022-12-31,2023-01-02,REGULAR TXN,NaN,NaN,NaN,NaN
3,'0239650517,31/12/2022,02/01/2023,"400,431.00",TRASPASO A CTA : 0664258739,504,4264,0.00,"2,625.58","756,359,740.52",...,02,01,2023,2022-12-31,2023-01-02,REGULAR TXN,NaN,NaN,NaN,NaN
4,'0239650517,31/12/2022,02/01/2023,"400,461.00",TRASPASO A CTA : 1018461782,504,4264,0.00,"1,993.14","756,357,747.38",...,02,01,2023,2022-12-31,2023-01-02,REGULAR TXN,NaN,NaN,NaN,NaN


In [16]:
edo_cta.shape

(12876303, 27)

In [17]:
edo_cta.columns

Index(['Cuenta', 'Fecha De Operación', 'Fecha', 'Referencia', 'Descripción',
       'Cod. Transac', 'Sucursal', 'Depósitos', 'Retiros', 'Saldo',
       'Movimiento', 'Descripción Detallada', 'Cheque', 'file_name',
       'Day_operacion', 'Month_operacion', 'Year_operacion', 'Day_banco',
       'Month_banco', 'Year_banco', 'Fecha_operacion', 'Fecha_banco', 'Tipo',
       'Afiliacion_1', 'len', 'Afiliacion_final', 'Tipo_operacion'],
      dtype='object')

In [18]:
edo_cta_limit = edo_cta[(edo_cta['Fecha_operacion'] >= start_date) & (edo_cta['Fecha_operacion'] <= final_date)]
edo_cta_limit['Fecha_operacion'].unique()

<DatetimeArray>
['2023-10-18 00:00:00', '2023-10-19 00:00:00', '2023-10-20 00:00:00',
 '2023-10-21 00:00:00', '2023-10-22 00:00:00', '2023-10-23 00:00:00',
 '2023-10-24 00:00:00', '2023-10-25 00:00:00', '2023-10-26 00:00:00',
 '2023-10-27 00:00:00', '2023-10-28 00:00:00', '2023-10-29 00:00:00',
 '2023-10-30 00:00:00']
Length: 13, dtype: datetime64[ns]

In [19]:
edo_cta_final = edo_cta_limit[['Cuenta','Fecha_operacion','Fecha_banco','Referencia','Descripción','Cod. Transac','Sucursal',
                               'Depósitos','Retiros','Saldo','Movimiento','Descripción Detallada','Cheque','Afiliacion_final',
                              'Tipo_operacion']]
edo_cta_final['Afiliacion_final'].fillna('not_found',inplace=True)
edo_cta_final['Tipo_operacion'].fillna('not_found',inplace=True)

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_2304\68710940.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edo_cta_final['Afiliacion_final'].fillna('not_found',inplace=True)
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_2304\68710940.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edo_cta_final['Tipo_operacion'].fillna('not_found',inplace=True)


In [20]:
edo_cta_final.head()

,Cuenta,Fecha_operacion,Fecha_banco,Referencia,Descripción,Cod. Transac,Sucursal,Depósitos,Retiros,Saldo,Movimiento,Descripción Detallada,Cheque,Afiliacion_final,Tipo_operacion
11257439,'0239650517,2023-10-18,2023-10-18,"20,231,017.00",COMISION X OPERACION CB,537,4264,0.00,260.50,"187,668,522.49",38711925,"OPERACIONES CONEXION BANORTE, OP 02: 000000002...",NaN,not_found,not_found
11257440,'0239650517,2023-10-18,2023-10-18,0.00,IVA DE COMISION X OPERACION CB,517,4264,0.00,41.68,"187,668,480.81",38711926,NaN,NaN,not_found,not_found
11257441,'0239650517,2023-10-18,2023-10-18,0.00,CLIPMX*SALONES DE BE 07927479C,060,302,"5,104.53",0.00,"187,673,585.34",38711927,No. de Pagos 24 Tipo de Plan 03 Periodo...,NaN,07927479,C
11257442,'0239650517,2023-10-18,2023-10-18,0.00,COMISION 07927479C,600,302,0.00,94.43,"187,673,490.91",38711928,No. de Pagos 24 Tipo de Plan 03 Periodo...,NaN,not_found,not_found
11257443,'0239650517,2023-10-18,2023-10-18,0.00,IVA COMISION 07927479C,601,302,0.00,15.11,"187,673,475.80",38711929,No. de Pagos 24 Tipo de Plan 03 Periodo...,NaN,not_found,not_found


In [21]:
edo_cta_final.shape

(475785, 15)

In [22]:
edo_cta_final_060 = edo_cta_final[(edo_cta_final['Cod. Transac'] == '060')]
edo_cta_final_060.head()

,Cuenta,Fecha_operacion,Fecha_banco,Referencia,Descripción,Cod. Transac,Sucursal,Depósitos,Retiros,Saldo,Movimiento,Descripción Detallada,Cheque,Afiliacion_final,Tipo_operacion
11257441,'0239650517,2023-10-18,2023-10-18,0.00,CLIPMX*SALONES DE BE 07927479C,060,302,"5,104.53",0.00,"187,673,585.34",38711927,No. de Pagos 24 Tipo de Plan 03 Periodo...,NaN,07927479,C
11257444,'0239650517,2023-10-18,2023-10-18,0.00,CLIPMX*SALONES DE BE 07927479C,060,302,"13,000.00",0.00,"187,686,475.80",38711930,No. de Pagos 18 Tipo de Plan 03 Periodo...,NaN,07927479,C
11257447,'0239650517,2023-10-18,2023-10-18,0.00,CLIPMX*SALONES DE BE 07927479C,060,302,"192,142.54",0.00,"187,878,339.36",38711933,No. de Pagos 12 Tipo de Plan 03 Periodo...,NaN,07927479,C
11257452,'0239650517,2023-10-18,2023-10-18,0.00,CLIPMX*SALONES DE BE 07927479C,060,302,"56,166.80",0.00,"187,924,098.73",38711938,No. de Pagos 09 Tipo de Plan 03 Periodo...,NaN,07927479,C
11257457,'0239650517,2023-10-18,2023-10-18,0.00,CLIPMX*SALONES DE BE 07927479C,060,302,"401,014.30",0.00,"188,323,490.08",38711943,No. de Pagos 06 Tipo de Plan 03 Periodo...,NaN,07927479,C


In [23]:
edo_cta_final_060.shape

(2477, 15)

In [24]:
edo_cta_final_060['Cod. Transac'].unique()

array(['060'], dtype=object)

In [25]:
edo_cta_final_060.to_excel('BNTE-517 COD_060 18-30 OCTUBRE 2023.xlsx',index=False)

# PIVOT CODIGO DE TRANSACCIÓN

In [ ]:
pt_cod_transc = pd.pivot_table(edo_cta_final,index=['Fecha_operacion'],
                    columns=['Cod. Transac'],
                    values=['Depósitos','Retiros']
                    ,aggfunc=np.sum,margins=True)
pt_cod_transc.fillna(0,inplace=True)
pt_cod_transc = pt_cod_transc.reset_index(drop=False)
# pt.to_csv('PIVOT BY EMAIL ABRIL.csv')
pt_cod_transc.head()

In [ ]:
pt_cod_transc.shape

In [ ]:
pt_cod_transc.to_excel('EDO_CTA_2023.xlsx')

# PIVOT AFILIACIÓN FINAL

In [ ]:
pt_afiliacion = pd.pivot_table(edo_cta_final_060,index=['Fecha_operacion'],
                    columns=['Afiliacion_final'],
                    values=['Depósitos','Retiros']
                    ,aggfunc=np.sum,margins=True)
pt_afiliacion.fillna(0,inplace=True)
pt_afiliacion = pt_afiliacion.reset_index(drop=False)
# pt.to_csv('PIVOT BY EMAIL ABRIL.csv')
pt_afiliacion.head()

In [ ]:
pt_afiliacion.shape

In [ ]:
pt_afiliacion.to_excel('EDO_CTA_2023_AFILIACION.xlsx')